In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import os

In [11]:
batch_size = 32 # Exemplos utilizados para um determinado processo
num_classes = 8 # classe de imagens possui 8 classes [Cachorros e Gatos]
dropout = 0.5
epochs = 500
lr = 0.001 #taxa de aprendizado ou otimização de erro

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/home/gregorio/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [12]:
base_model = tf.keras.applications.VGG19(weights='imagenet' )
x=base_model.output
#x=tf.keras.layers.Flatten()(x)
#x=tf.keras.layers.GlobalAveragePooling2D()(x)

#adiciona apos x duas camada densa com 4096 neuronios com funcao de ativacao relu. Atribui este no a x novamente
#x=tf.keras.layers.Dense(4096,activation='relu')(x)
#x=tf.keras.layers.Dense(4096,activation='relu')(x)

#adiciona apos x uma camada densa com 8 neuronios (oito classes) com funcao de ativacao softmax (distribuicao de probabilidade). Atribui este no a preds
preds=tf.keras.layers.Dense(8,activation='softmax', name='prediction')(x)

#definindo modelo final
model=tf.keras.models.Model(inputs=base_model.input,outputs=preds)

for l in model.layers:
  if l.name.split('_')[0] != 'prediction':
    l.trainable=True
  else:
    l.trainable=False
    
#mostrando modelo final e sua estrutura
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
#inicializando o objeto que ira recuperar as amostras de treino com a funcao de pre-processamento da vgg19
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg19.preprocess_input) #included in our dependencies

#inicializando o objeto que ira recuperar as amostras de teste com a funcao de pre-processamento da vgg19
test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg19.preprocess_input)

#definindo a origem das imagens de treino. Este ponto de origem deve possuir pastas separando as amostras em classes, onde os nomes das pastas devem ser os mesmos das classes [TREINO]
train_generator = train_data_gen.flow_from_directory('/home/gregorio/Downloads/archive/data/natural_images',
                                                 target_size=(28,28),
                                                 batch_size=batch_size,
                                                 shuffle=True)

#definindo a origem das imagens de test. Este ponto de origem deve possuir pastas separando as amostras em classes, onde os nomes das pastas devem ser os mesmos das classes [TESTE]
test_generator = train_data_gen.flow_from_directory('/home/gregorio/Downloads/archive/natural_images',
                                                 target_size=(28,28),
                                                 batch_size=16,
                                                 shuffle=True)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

Found 5521 images belonging to 8 classes.
Found 1378 images belonging to 8 classes.


In [ ]:
step_size_train= train_generator.n//train_generator.batch_size
step_size_test= test_generator.n//test_generator.batch_size

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=500,
                    validation_data=test_generator,
                    validation_steps=step_size_test)

Epoch 1/500


In [1]:
path_train='/home/gregorio/Downloads/archive/data/natural_images'
path_test ='/home/gregorio/Downloads/archive/natural_images'



#Avaliando o modelo
print('Accuracy')
loss_train, train_acc = model.evaluate_generator(train_generator, steps=step_size_train)
loss_test, test_acc = model.evaluate_generator(test_generator, steps=step_size_test)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
#Apresentando resultados em graficos
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label="test")
plt.legend()
plt.show()


# Criando graficos para visualização dos resultados
print("\n\n")
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label="train")
plt.plot(history.history['val_accuracy'], label="test")
plt.legend()
plt.show()

print('Criando classificações..')
labels = os.listdir(path_train)
print('Rótulos', labels)

#criando estruturas para métricas de avaliação, processo um pouco mais demorado
Y_pred = model.predict_generator(test_generator)
print('Preds Created')
y_pred = np.argmax(Y_pred, axis=1)
print('Preds 1D created')

classification = classification_report(test_generator.classes, y_pred, target_names=labels)
print('----------------CLASSIFICATION--------------')
print(classification)
matrix = confusion_matrix(test_generator.classes, y_pred)
df_cm = pd.DataFrame(matrix, index = [i for i in range(8)],
                  columns = [i for i in range(8)])
fig2 = plt.figure(figsize = (10,7))
fig2.suptitle('Matriz de confusão')
sns.heatmap(df_cm, annot=True, linewidths=2.5)

Accuracy


NameError: name 'model' is not defined